## TensorFlow Tutorial

* Tensor : a expansion of vector and matrix
* Zero-order tensor : Scalar
* First-order tensor : Vector
* Second-order tensor : Matrix
* Tensor can be represented as a N-Dimension array


### Numpy


In [2]:
import numpy as np

a = np.zeros((2, 2));
b = np.ones((2, 2));

np.sum(b, axis=1)

array([ 2.,  2.])

In [3]:
a.shape 

(2, 2)

In [4]:
np.reshape(a, (1, 4))

array([[ 0.,  0.,  0.,  0.]])

### Tensorflow

In [5]:
import tensorflow as tf

sess = tf.Session()

a = tf.zeros((2, 2))
a = tf.ones((2, 2))

sess.run(tf.reduce_sum(b, axis=1))

array([ 2.,  2.])

In [6]:
a.get_shape()

TensorShape([Dimension(2), Dimension(2)])

In [7]:
sess.run(tf.reshape(a, (1, 4)))

array([[ 1.,  1.,  1.,  1.]], dtype=float32)

In [9]:
sess = tf.Session()
a = np.zeros((2, 2));
ta = tf.zeros((2, 2)); 
print(a)   # numpy 는 바로 배열이 출력됨

[[ 0.  0.]
 [ 0.  0.]]


In [10]:
print(ta) # tensorflow 는 tensor 가 출력

Tensor("zeros_1:0", shape=(2, 2), dtype=float32)


In [12]:
print(sess.run(ta)) # 배열 형태로 출력하기위해서는 tensor 를 실행해 출력

[[ 0.  0.]
 [ 0.  0.]]


### Computation Graph


In [14]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b # Computation graph -> c 는 tensor

print(c)
sess = tf.Session() 
print(sess.run(c)) 

Tensor("mul_1:0", shape=(), dtype=float32)
30.0


### Tensorflow Session


In [16]:
sess = tf.Session() # session 생성 방법 1
print(sess.run(c))

with tf.Session() as sess: # session 생성 방법 2
    print(sess.run(c))
    print(c.eval()) # 해당 범위 내에 위치할 때만 유효

30.0
30.0
30.0


### Tensorflow Variables

* 모든 tensor 는 기본적으로 constant tensor
* tensor 의 값을 지정하고 변경하기 위해 variable type 사용
* variable type 은 반드시 초기화(initialization) 이 필요

In [20]:
# get error!!

w = tf.Variable(tf.zeros((2, 2)), name="weight")
with tf.Session() as sess:
    print(sess.run(w))

FailedPreconditionError: Attempting to use uninitialized value weight_3
	 [[Node: _send_weight_3_0 = _Send[T=DT_FLOAT, client_terminated=true, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=-1291397192728678093, tensor_name="weight_3:0", _device="/job:localhost/replica:0/task:0/cpu:0"](weight_3)]]

* variable initialization 

In [22]:
w = tf.Variable(tf.random_normal([5, 2], stddev=0.1), name="weight")
# constant or ramdom 값으로 초기화 가능

with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) 
    print(sess.run(w))

[[-0.09221601  0.08975536]
 [-0.00838273 -0.22405207]
 [-0.09514918 -0.04961732]
 [-0.02487294  0.08336319]
 [ 0.0438805   0.07881008]]


* variable update

In [23]:
state = tf.Variable(0, name="counter") 
new_value = tf.add(state, tf.constant(1))
update = tf.assign(state, new_value)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(state))
    for _ in range(3) : 
        sess.run(update) 
        print(sess.run(state))

0
1
2
3


* variable fetch

In [25]:
x1 = tf.constant(1)
x2 = tf.constant(2)
x3 = tf.constant(3)
temp = tf.add(x2, x3) 
mul = tf.multiply(x1, temp) 


with tf.Session() as sess: 
    result1, result2 = sess.run([mul, temp])
    print(result1, result2)

5 5


### Tensorflow Placeholder

In [26]:
a = tf.placeholder(tf.int16) 
b = tf.placeholder(tf.int16)

add = tf.add(a, b)
mul = tf.multiply(a, b) 

with tf.Session() as sess:
    print(sess.run(add, feed_dict={a: 2, b: 3})) 
    print(sess.run(mul, feed_dict={a: 2, b: 3})) 

5
6


In [28]:
# using tf.constant
matrix1 = tf.constant([[3., 3.]]) 
matrix2 = tf.constant([[2.], [2.]])
product = tf.matmul(matrix1, matrix2)

with tf.Session() as sess: 
    result = sess.run(product) 
    print(result)
    
# using placeholder
import numpy as np

matrix1 = tf.placeholder(tf.float32, [1, 2]) 
matrix2 = tf.placeholder(tf.float32, [2, 1])
product = tf.matmul(matrix1, matrix2) 

with tf.Session() as sess: 
    mv1 = np.array([[3., 3.]])
    mv2 = np.array([[2.], [2.]])
    result = sess.run(product, feed_dict={matrix1: mv1, matrix2: mv2}) 
    print(result)

[[ 12.]]
[[ 12.]]


### Example - MNIST with MLP

In [37]:
import tensorflow as tf
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True) # MNIST 데이터셋 불러오기

learning_rate = 0.001
max_steps = 15000
batch_size = 128

x = tf.placeholder(tf.float32, [None, 784]) 
y = tf.placeholder(tf.float32, [None, 10])

def MLP(inputs):
    W_1 = tf.Variable(tf.random_normal([784, 256]))
    b_1 = tf.Variable(tf.zeros([256]))
    
    W_2 = tf.Variable(tf.random_normal([256, 256]))
    b_2 = tf.Variable(tf.zeros([256]))
    
    W_out = tf.Variable(tf.random_normal([256, 10]))
    b_out = tf.Variable(tf.zeros([10]))
    
    h_1 = tf.add(tf.matmul(inputs, W_1), b_1)
    h_1 = tf.nn.relu(h_1)
    
    h_2 = tf.add(tf.matmul(h_1, W_2), b_2)
    h_2 = tf.nn.relu(h_2)
    
    out = tf.add(tf.matmul(h_2, W_out), b_out)
    
    return out

net = MLP(x)

# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=net, labels=y))
opt = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)

# initializing the variables
init_op = tf.global_variables_initializer() 

sess = tf.Session()
sess.run(init_op)

# train model
for step in range(max_steps):
    batch_X, batch_y = mnist.train.next_batch(batch_size)
    _, loss = sess.run([opt, loss_op], feed_dict={x: batch_X, y: batch_y})
    
    if(step+1) % 1000 == 0:
        print("[{}/{}] loss:{:.3f}".format(step+1, max_steps, loss))
print("Optimaization Finished!")

# test model
correct_prediction = tf.equal(tf.argmax(net, 1), tf.argmax(y, 1)) 

# calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Train accuracy: {:.3f}".format(sess.run(accuracy, feed_dict={x: mnist.train.images, y: mnist.train.labels})))
print("Test accuracy: {:.3f}".format(sess.run(accuracy, feed_dict={x: mnist.test.images, y: mnist.test.labels})))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
[1000/15000] loss:38.269
[2000/15000] loss:9.517
[3000/15000] loss:18.316
[4000/15000] loss:1.633
[5000/15000] loss:5.048
[6000/15000] loss:3.119
[7000/15000] loss:0.131
[8000/15000] loss:1.537
[9000/15000] loss:0.684
[10000/15000] loss:0.121
[11000/15000] loss:0.000
[12000/15000] loss:0.000
[13000/15000] loss:0.426
[14000/15000] loss:0.000
[15000/15000] loss:0.000
Optimaization Finished!
Train accuracy: 0.995
Test accuracy: 0.952


### Variable Scope
* tf.variable_scope() : 네임스페이스 관리(prefix)
* tf.get_variable() : 변수 생성 및 반환

* tf.variable_scope()

In [39]:
var1 = tf.Variable([1], name="var") 
with tf.variable_scope("foo"): 
    with tf.variable_scope("bar"):
        var2 = tf.Variable([1], name="var") 
        var3 = tf.Variable([1], name="var")
        
print ("var1: {}".format(var1.name))
print ("var2: {}".format(var2.name))
print ("var3: {}".format(var3.name))

var1: var_1:0
var2: foo_1/bar/var:0
var3: foo_1/bar/var_1:0


* tf.get_variable()

In [41]:

var1 = tf.Variable([1], name="var")
with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var2 = tf.Variable([1], name="var")
        scp.reuse_variables() # allow reuse variables
        var3 = tf.Variable([1], name="var")
        
print ("var1: {}".format(var1.name))
print ("var2: {}".format(var2.name))
print ("var3: {}".format(var3.name))

var1: var_3:0
var2: foo_3/bar/var:0
var3: foo_3/bar/var_1:0


* Parameter sharing

In [42]:

with tf.variable_scope("foo"):
    with tf.variable_scope("bar") as scp:
        var1 = tf.get_variable("var", [1])
        scp.reuse_variables() # allow reuse variables
        var2 = tf.get_variable("var", [1])
        
    with tf.variable_scope("bar", reuse=True): 
        var3 = tf.get_variable("var", [1])
        
print ("var1: {}".format(var1.name))
print ("var2: {}".format(var2.name))
print ("var3: {}".format(var3.name))

var1: foo/bar/var_2:0
var2: foo/bar/var_2:0
var3: foo/bar/var_2:0


- tf.Variable 대신 tf.get_variable 사용 해서, parameter sharing 
- scoping 에 주의

### Example - MNIST with CNN

In [43]:
# Wrappers 


# Another init strategy called He_init instoduced in PReLUNet
from tensorflow.contrib.layers import variance_scaling_initializer
he_init = variance_scaling_initializer()

def conv(bottom, num_filter, ksize=3, stride=1, padding="SAME", scope=None):
    bottom_shape = bottom.get_shape().as_list()[3] # To calculate shape of previous (input) layer
    
    with tf.variable_scope(scope or "conv"):
        W = tf.get_variable("W", [ksize, ksize, bottom_shape, num_filter], initializer=he_init)
        b = tf.get_variable("b", [num_filter], initializer=tf.constant_initializer(0))
        x = tf.nn.conv2d(bottom, W, strides=[1, stride, stride, 1], padding=padding)
        x = tf.nn.relu(tf.nn.bias_add(x, b)) 
        
    return x


def maxpool(bottom, ksize=2, stride=2, padding="SAME", scope=None):
    with tf.variable_scope(scope or "maxpool"):
        pool = tf.nn.max_pool(bottom, ksize=[1, ksize, ksize, 1], strides=[1, stride, stride, 1], padding=padding)
    
    return pool


def fc(bottom, num_dims, scope=None):
    bottom_shape = bottom.get_shape().as_list()
    if len(bottom_shape) > 2:
        bottom = tf.reshape(bottom, [-1, reduce(lambda x, y: x*y, bottom_shape[1:])])
        bottom_shape = bottom.get_shape().as_list()
        
    with tf.variable_scope(scope or "fc"):
        W = tf.get_variable("W", [bottom_shape[1], num_dims], initializer=he_init)
        b = tf.get_variable("b", [num_dims], initializer=tf.constant_initializer(0))
        out = tf.nn.bias_add(tf.matmul(bottom, W), b)
    
    return out

def fc_relu(bottom, num_dims, scope=None):
    with tf.variable_scope(scope or "fc"):
        out = fc(bottom, num_dims, scope="fc")
        relu = tf.nn.relu(out)
        
    return relu

# All Together

keep_prob = tf.placeholder(tf.float32, None)

def conv_net(x, keep_prob):
    x = tf.reshape(x, shape=[-1, 28, 28, 1]) 
    
    conv1 = conv(x, 32, 5, scope="conv_1")
    conv1 = maxpool(conv1, scope="maxpool_1") 
    conv2 = conv(conv1, 64, 5, scope="conv_2")
    conv2 = maxpool(conv2, scope="maxpool_2")
    
    fc1 = fc_relu(conv2, 1024, scope="fc_1")
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    out = fc(fc1, 10, scope="out")
    return out

### Layers

In [52]:
x = tf.placeholder("float")
input = tf.reshape(x, [3, 3, 3, 64]) 

with tf.name_scope('conv1_1') as scope:
    kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32, stddev=1e-1), name='weights')
    conv = tf.nn.conv2d(x, kernel, [1, 1, 1, 1], padding='SAME')
    biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32), trainable=True, name='biases')
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope)

* TF-Slim Code

In [53]:
import tensorflow.contrib.slim as slim

net = slim.conv2d(input, 128, [3, 3], padding='SAME', scope='conv1_1')

x = tf.placeholder("float")

# 1. simple network generation with slim
net = tf.reshape(x, [3, 3, 3, 64]) 

net = slim.conv2d(net, 256, [3, 3], scope='conv3_1')
net = slim.conv2d(net, 256, [3, 3], scope='conv3_2')
net = slim.conv2d(net, 256, [3, 3], scope='conv3_3')
net = slim.max_pool2d(net, [2, 2], scope='pool3')

# 1. cleaner by repeat operation:
net = tf.reshape(x, [3, 3, 3, 64])
net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3], scope='conv3')
net = slim.max_pool(net, [2, 2], scope='pool3')

# 2. Verbose way:
x = slim.fully_connected(x, 32, scope='fc/fc_1')
x = slim.fully_connected(x, 64, scope='fc/fc_2')
x = slim.fully_connected(x, 128, scope='fc/fc_3')

# 2. Equivalent, TF-Slim way using slim.stack:
slim.stack(x, slim.fully_connected, [32, 64, 128], scope='fc')

ValueError: Trying to share variable conv1_1/weights, but specified shape (3, 3, 64, 128) and found shape (3, 3, 3, 128).